In [1]:
import os
from openai import OpenAI
from pydantic import BaseModel
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool, WebSearchTool
import sendgrid
import asyncio
from sendgrid.helpers.mail import Mail, Email, To, Content
from IPython.display import display, Markdown
from typing import Dict
from agents.model_settings import ModelSettings

In [2]:
load_dotenv(override=True)

True

In [25]:
instructions1="You are a research assistant. You will be given a word. You are search the web for that term \
    produce a concise summary of the results. The summary must be of 1 paragraph and should not exceed 50 \
    words."

search_agent = Agent(
    name="Search Agent",
    instructions=instructions1,
    model=os.getenv("model"),
    tools= [WebSearchTool(search_context_size="low")],
    model_settings=ModelSettings(tool_choice="required")
)

In [6]:
message = "Latest AI Agentic framewords 2025"

with trace("Agentic Frameworks"):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))

As of June 2025, several AI agentic frameworks have emerged, enhancing the development of autonomous systems capable of complex decision-making and collaboration. Microsoft's AutoGen orchestrates multi-agent systems with an event-driven architecture, facilitating scalable and efficient AI workflows. LangChain enables seamless AI workflows by chaining prompts, memory, and tools, while LangGraph offers advanced state management through graph-based architectures, ideal for applications requiring dependency management. Microsoft's Semantic Kernel bridges semantic understanding with functionality, enhancing contextual decision-making in AI applications. CrewAI focuses on multi-agent collaboration, allowing for task execution by multiple agents, and Eliza provides a Web3-friendly AI agent operating system, integrating blockchain data and smart contracts into AI functionalities. ([lekha-bhan88.medium.com](https://lekha-bhan88.medium.com/top-5-agentic-ai-frameworks-to-watch-in-2025-9d51b2b652c0?utm_source=openai), [arxiv.org](https://arxiv.org/abs/2501.06781?utm_source=openai))

In November 2024, Anthropic introduced the Model Context Protocol (MCP), an open-source framework standardizing the integration of AI models with external tools and data sources. MCP aims to serve as a universal connector between language-model agents and external software, facilitating secure and efficient data exchange. Its adoption by major AI providers, including OpenAI and Google DeepMind, underscores its significance in the AI community. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Model_Context_Protocol?utm_source=openai))


## Recent Developments in AI Agentic Frameworks:
- [Nvidia GTC 2025 - all the news you might have missed](https://www.techradar.com/pro/live/nvidia-gtc-2025-all-the-news-and-updates-from-jensen-huang-keynote-as-it-happens?utm_source=openai)
- [Microsoft Build 2025 LIVE: All the big AI news announced](https://www.tomsguide.com/news/live/microsoft-build-2025?utm_source=openai)
- [5 Predictions for AI in 2025](https://time.com/7204665/ai-predictions-2025/?utm_source=openai) 

In [31]:
HOW_MANY_SEARCHES = 1

instructions2 = f"You will be given a term and you are supposed to provide the query to enter on the \
    internet to get the best possible answer. Provide me with {HOW_MANY_SEARCHES} queries."

class WebSearchItem(BaseModel):
    reason: str
    "Your reason for this particular query"

    query: str
    "The query that i should search"

class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem]
    """A List of Web Searches to get the best answer for the query"""

planner_agent = Agent(
    name="Planner Agent",
    model = os.getenv("model"),
    instructions=instructions2,
    output_type=WebSearchPlan
)

In [17]:
message = "Cricketers with the best Wicketkeeping techniques"

with trace("Cricketers"):
    result = await Runner.run(planner_agent, message)
    print(result.final_output)

search=[WebSearchItem(reason='To find detailed insights and analyses of cricketers known for their wicketkeeping skills.', query='best wicketkeepers in cricket history techniques'), WebSearchItem(reason='To find current players recognized for their wicketkeeping abilities and strategies for success.', query='top modern wicketkeepers cricket techniques')]


In [2]:
@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    """Send out an email with the given subject and HTML body"""
    try:
        sg = sendgrid.SendGridAPIClient(api_key=os.getenv("SENDGRID_API_KEY"))  # <-- corrected!
        from_email = Email("22bce338@nirmauni.ac.in")
        to_email = To("siddhs2004@gmail.com")
        content = Content("text/html", html_body)
        mail = Mail(from_email, to_email, subject, content).get()

        response = sg.client.mail.send.post(request_body=mail)

        print("✅ SendGrid API Response Code:", response.status_code)
        print("📩 Response Body:", response.body)

        if response.status_code == 202:
            return {"status": "success"}
        else:
            return {
                "status": "failed",
                "code": response.status_code,
                "body": response.body.decode("utf-8") if hasattr(response.body, "decode") else str(response.body)
            }

    except Exception as e:
        print("❌ Exception occurred while sending email:")
        print(str(e))
        return {"status": "error", "message": str(e)}


In [20]:
instructions3 = """You are able to send a nicely formatted HTML email based on a detailed report.
You will be provided with a detailed report. You should use your tool to send one email, providing the 
report converted into clean, well presented HTML with an appropriate subject line."""

email_agent = Agent(
    name = "Email Agent",
    instructions=instructions3,
    tools=[send_email],
    model = os.getenv("model")
)

In [21]:
instructions4 = "You are a report writing agent. You will be given a query and a short summary of the \
    of the findings of the query. Your task is to generate a report on it. The report should not contain \
    more than 100 words. You should first generate the flow of the report and then output the final report."

class ReportData(BaseModel):
    short_summary: str
    "A short summary of the findings"

    actual_report: str
    "The Actual report to be written"

    follow_up: str
    "Any follow up questions and recommendations to work on a future topic"

writer_agent = Agent(
    name= "Writer Agent",
    instructions=instructions4,
    model = os.getenv("model"),
    output_type=ReportData,
)

In [24]:
async def plan_searches(query: str):
    """Use the planner agent to find the relevant search wordings"""
    print("Planning Searches")
    result = await Runner.run(planner_agent, f"Query {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

async def search(item: WebSearchItem):
    """Use search agent to search each query"""
    input = f"Search {item.query} and the reason {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output

async def all_searches(search_plan : WebSearchPlan):
    """Use this to search all the web searches"""
    print("Searching....")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished Searching")
    return results

In [52]:
async def write_report(query: str, search_results: list[str]):
    """Use the writer agent to write the report"""
    print("Writing a report...")
    input = f"Query is: {query} and the summary is: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Report Written")
    return result.final_output

async def send_email(report: ReportData):
    """ Use the email agent to send an email with the report """
    print("Writing email...")
    result = await Runner.run(email_agent, report.actual_report)
    print("Email sent")
    return report


: 

In [ ]:
query = "Why is MS Dhoni better than Virat Kohli"

with trace("Cricketing Questions"):
    search_plan = await plan_searches(query)
    search_results = await all_searches(search_plan)
    final_report = await write_report(query, search_results)
    await send_email(final_report)
    print("Lessssgooooo!!")


Planning Searches
Will perform 1 searches
Searching....
Finished Searching
Writing a report...
Report Written
Writing email...
Email sent
RunResult:
- Last agent: Agent(name="Email Agent", ...)
- Final output (str):
    It seems there's an error preventing me from sending the email at the moment. However, you can take the formatted HTML content and send it through your own email client. Here's the HTML content you can use:
    
    ```html
    <h1>Comparative Analysis of Indian Cricket Captains</h1>
    <p>India boasts two of the most celebrated cricket captains: <strong>MS Dhoni</strong> and <strong>Virat Kohli</strong>, each contributing uniquely to the sport.</p>
    
    <h2>MS Dhoni: The Calm Strategist</h2>
    <ul>
      <li><strong>Leadership Style:</strong> Characterized by composure and strategic man-management.</li>
      <li><strong>Achievements:</strong> Led India to three ICC trophies: 
        <ul>
          <li>T20 World Cup</li>
          <li>ODI World Cup</li>
       